In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error,mean_absolute_error

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

from utils.utils import *
from utils.constants import *

# Data

To make valid comparison across different methods, we split the original `stack_train` into new train and validation data sets.

In [3]:
# Import data


y_train = pd.read_csv(get_absolute_path('y_train.csv', 'data'))
y_test  = pd.read_csv(get_absolute_path('y_test.csv', 'data'))


stack_train = pd.read_csv(get_absolute_path('stacked_X_tr.csv', 'data'))
stack_test  = pd.read_csv(get_absolute_path('stacked_X_te.csv', 'data'))



In [4]:
stack_train = stack_train.astype(column_data_extended_types)
stack_test = stack_test.astype(column_data_extended_types)

In [5]:
stack_train.head()

,"Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Maximum)","pH, water, unfiltered, field, standard units (Maximum)","pH, water, unfiltered, field, standard units (Minimum)","Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Minimum)","Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Mean)","Dissolved oxygen, water, unfiltered, milligrams per liter (Maximum)","Dissolved oxygen, water, unfiltered, milligrams per liter (Mean)","Dissolved oxygen, water, unfiltered, milligrams per liter (Minimum)","Temperature, water, degrees Celsius (Mean)","Temperature, water, degrees Celsius (Minimum)","Temperature, water, degrees Celsius (Maximum)",Date,Location_ID,Month,Week,Weekday
0,0.001131,0.884615,0.001120,0.001113,0.677632,0.841463,0.765152,0.787402,0.293750,0.298077,0.276163,2016-01-28,2198840.0,1,4,3
1,0.001170,0.871795,0.001159,0.001152,0.703947,0.829268,0.772727,0.795276,0.293750,0.301282,0.276163,2016-01-28,2198920.0,1,4,3
2,0.001326,0.884615,0.001198,0.001250,0.677632,0.853659,0.750000,0.755906,0.300000,0.298077,0.287791,2016-01-28,2198950.0,1,4,3
3,0.014094,0.858974,0.001238,0.003926,0.697368,0.829268,0.772727,0.771654,0.296875,0.294872,0.279070,2016-01-28,2203603.0,1,4,3
4,0.088109,0.858974,0.010766,0.029297,0.684211,0.853659,0.765152,0.755906,0.296875,0.291667,0.281977,2016-01-28,2203655.0,1,4,3


# Feature Engineering

In [6]:
# Select numeric and categorical columns
numeric_columns = stack_train.select_dtypes(include=['float64']).columns
categorical_columns = [#'Date', 
                       'Location_ID',
                    #    'Year',
                       'Month',
                       'Week',
                       'Weekday']  # Add any categorical columns here

# Create preprocessing transformers
numeric_transformer = StandardScaler()  # we can use other scalers as well
categorical_transformer = OneHotEncoder(drop=None)  # Use one-hot encoding for categorical columns

# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

# Fit the preprocessor on training data and transform both train and test data
X_train_preprocessed = preprocessor.fit_transform(stack_train)
X_test_preprocessed  = preprocessor.transform(stack_test)


# Get the column names after one-hot encoding
categorical_encoded_columns = preprocessor.named_transformers_['cat']\
                                    .get_feature_names_out(input_features=categorical_columns)

# Convert X_train_preprocessed and X_test_preprocessed to DataFrames

X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed.toarray(), columns=np.concatenate([numeric_columns, categorical_encoded_columns]))
X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed.toarray(), columns=np.concatenate([numeric_columns, categorical_encoded_columns]))


In [7]:
X_train_preprocessed_df.columns

Index(['Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Maximum)',
       'pH, water, unfiltered, field, standard units (Maximum)',
       'pH, water, unfiltered, field, standard units (Minimum)',
       'Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Minimum)',
       'Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Mean)',
       'Dissolved oxygen, water, unfiltered, milligrams per liter (Maximum)',
       'Dissolved oxygen, water, unfiltered, milligrams per liter (Mean)',
       'Dissolved oxygen, water, unfiltered, milligrams per liter (Minimum)',
       'Temperature, water, degrees Celsius (Mean)',
       'Temperature, water, degrees Celsius (Minimum)',
       ...
       'Week_7', 'Week_8', 'Week_9', 'Weekday_0', 'Weekday_1', 'Weekday_2',
       'Weekday_3', 'Weekday_4', 'Weekday_5', 'Weekday_6'],
      dtype='object', length=119)

# XGBoost

Adding hyperparameter tuning.

- RMSE based
- MAE based

In [8]:
##### RMSE based

# Define XGBoost parameters grid for tuning
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200, 500],
    'subsample': [0.6, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5, 1, 1.5, 2, 5],
    'min_child_weight': [1, 3, 5, 10, 20, 100]
}

# Create an XGBoost model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror')
# model_xgb_mae = xgb.XGBRegressor(objective='reg:linear')

# Define a custom scoring function (negative RMSE since GridSearchCV minimizes the score)
scoring = make_scorer(lambda y_true, y_pred: -mean_squared_error(y_true, y_pred, squared=False))


# Perform hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(model_xgb, param_grid, cv=5, scoring=scoring)
grid_search.fit(X_train_preprocessed, y_train)

# Get the best hyperparameters and best model
best_xgb_params = grid_search.best_params_
best_xgb_model = grid_search.best_estimator_
best_xgb_score = grid_search.best_score_

print("Best Hyperparameters:", best_xgb_params)
print("Best RMSE:", -best_xgb_score)

# Making predictions on the validation data using the best model
y_pred_xgb = best_xgb_model.predict(X_test_preprocessed)

# Calculating RMSE on the validation data
rmse_xgb = mean_squared_error(y_test, y_pred_xgb, squared=False)
print("XGBoost RMSE on Validation Data with Best Model:", rmse_xgb)


# Get feature importance scores
xgb_feature_importance = best_xgb_model.feature_importances_

# Create a list of feature names
feature_names = X_train_preprocessed_df.columns

# Create a dictionary mapping feature names to their importance scores
xgb_feature_importance_dict = dict(zip(feature_names, xgb_feature_importance))

# Sort feature importance scores in descending order
xgb_sorted_feature_importance = sorted(xgb_feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print feature importance scores
print("Feature Importance:")
for feature, importance in xgb_sorted_feature_importance:
    print(f"{feature}: {importance}")


best_xgb_model_info = {'best_params': best_xgb_params, 'best_score': best_xgb_score}

# Create a dictionary containing the feature importance results
feature_importance_dict = dict(xgb_sorted_feature_importance)

# Add the feature importance dictionary to best_xgb_model_info
best_xgb_model_info['feature_importance'] = feature_importance_dict



best_xgb_file = get_absolute_path(
    file_name = 'best_xgb_model.joblib'
    , rel_path = 'results'
)

# Save the updated best_xgb_model_info using save_model function
save_model(best_xgb_file, best_xgb_model, best_xgb_model_info)

# # Load the model and its info
# best_xgb_file = get_absolute_path(
#     file_name = 'best_xgb_model.joblib'
#     , rel_path = 'results'
# )
# best_xgb_model, best_xgb_model_info = load_model(best_xgb_file)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1465603777.py, line 8)

In [ ]:
### MAE based


# Create an XGBoost model with 'reg:linear' objective
model_xgb_mae = xgb.XGBRegressor(objective='reg:linear')

# Define a custom scoring function (negative MAE since GridSearchCV minimizes the score)
scoring_mae = make_scorer(lambda y_true, y_pred: -mean_absolute_error(y_true, y_pred))

# Perform hyperparameter tuning using GridSearchCV
grid_search_mae = GridSearchCV(model_xgb_mae, param_grid, cv=5, scoring=scoring_mae)
grid_search_mae.fit(X_train_preprocessed_df, y_train)

# Get the best hyperparameters and best model
best_xgb_params_mae = grid_search_mae.best_params_
best_xgb_model_mae = grid_search_mae.best_estimator_
best_xgb_score_mae = grid_search_mae.best_score_

print("Best Hyperparameters (MAE):", best_xgb_params_mae)
print("Best MAE:", -best_xgb_score_mae)

# Making predictions on the validation data using the best model
y_pred_xgb_mae = best_xgb_model_mae.predict(X_test_preprocessed_df)

# Calculating MAE on the validation data
mae_xgb_mae = mean_absolute_error(y_test, y_pred_xgb_mae)
print("XGBoost MAE on Validation Data with Best Model:", mae_xgb_mae)

# Get feature importance scores
xgb_feature_importance_mae = best_xgb_model_mae.feature_importances_

# Create a list of feature names
feature_names_mae = X_train_preprocessed_df.columns

# Create a dictionary mapping feature names to their importance scores
xgb_feature_importance_dict_mae = dict(zip(feature_names_mae, xgb_feature_importance_mae))

# Sort feature importance scores in descending order
xgb_sorted_feature_importance_mae = sorted(xgb_feature_importance_dict_mae.items(), key=lambda x: x[1], reverse=True)

# Print feature importance scores
print("Feature Importance (MAE):")
for feature, importance in xgb_sorted_feature_importance_mae:
    print(f"{feature}: {importance}")

best_xgb_model_info_mae = {'best_params': best_xgb_params_mae, 'best_score': best_xgb_score_mae}

# Create a dictionary containing the feature importance results
feature_importance_dict_mae = dict(xgb_sorted_feature_importance_mae)

# Add the feature importance dictionary to best_xgb_model_info_mae
best_xgb_model_info_mae['feature_importance'] = feature_importance_dict_mae

best_xgb_file_mae = 'best_xgb_model_mae.joblib'

# Save the updated best_xgb_model_info_mae using save_model function
# save_model(best_xgb_file_mae, best_xgb_model_mae, best_xgb_model_info_mae)

# Load the model and its info
# best_xgb_model_mae, best_xgb_model_info_mae = load_model(best_xgb_file_mae)

[22:57:31] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:57:31] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.

[22:57:31] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[22:57:31] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/learner.cc:767: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.

[22:57:32] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/objective/regression_obj.

In [ ]:
# Calculating MAE on the validation data
mae_xgb_mae = mean_absolute_error(y_test, y_pred_xgb_mae)
print("XGBoost MAE on Validation Data with Best Model:", mae_xgb_mae)

XGBoost MAE on Validation Data with Best Model: 0.0064234025687788954


### Note:

When we don't convert our data to xgb.DMatrix, we can use tools like GridSearchCV directly from scikit-learn. This is because scikit-learn's GridSearchCV works with the standard array-like data formats that scikit-learn supports, such as NumPy arrays and Pandas DataFrames.

When we use xgb.DMatrix, we're utilizing XGBoost's own data structure that is optimized for performance, memory usage, and compatibility with XGBoost's specific features. However, this data structure is not natively understood by scikit-learn's utilities like GridSearchCV.

If we decide not to convert our data to xgb.DMatrix and we want to use GridSearchCV, we can proceed with our original code. The scikit-learn grid search will work directly with our DataFrame and Series data.

In [ ]:
# import shap

# # Wrap the XGBoost model in a function
# def xgb_predictor(data):
#     return best_xgb_model.predict(data)

# # Initialize a SHAP explainer with the predictor function
# xgb_explainer = shap.Explainer(xgb_predictor, data=dval)

# # Calculate SHAP values for a set of data (e.g., dval)
# xgb_shap_values = xgb_explainer.shap_values(dval)

# # Create a summary plot of feature importances using SHAP
# shap.summary_plot(xgb_shap_values, dval, plot_type="bar")


In [ ]:
# best_xgb_model

# _xgb_model = best_xgb_model.fit(dtrain, y_train)

# # explain the model's predictions using SHAP
# # (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
# _xgb_explainer = shap.Explainer(_xgb_model)
# _xgb_shap_values = _xgb_explainer(dtrain)

# # visualize the first prediction's explanation
# shap.plots.waterfall(_xgb_shap_values[0])

# Random Forest

In [ ]:
# Define RandomForest parameters grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200, 500],
    'max_depth': [None, 3, 5, 10, 20],
    'min_samples_split': [1, 2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a RandomForest model
model_rf = RandomForestRegressor()


# Define a custom scoring function (negative RMSE since GridSearchCV minimizes the score)
scoring = make_scorer(lambda y_true, y_pred: -mean_squared_error(y_true, y_pred, squared=False))

# Perform hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(model_rf, param_grid, cv=5, scoring=scoring)
# grid_search.fit(dtrain, y_train)
grid_search.fit(X_train_preprocessed, y_train)


# Get the best hyperparameters and best model
best_rf_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_
best_rf_score = -grid_search.best_score_

print("Best Hyperparameters:", best_rf_params)

# Making predictions on the validation data using the best model
y_pred_rf = best_rf_model.predict(dval)

# Calculating RMSE on the validation data
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
print("RandomForest RMSE on Validation Data with Best Model:", rmse_rf)

# Get feature importance scores for the best Random Forest model
rf_feature_importance = best_rf_model.feature_importances_

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = {feature_name: importance_score for feature_name, importance_score in zip(X_train_preprocessed_df.columns, rf_feature_importance)}

# Sort feature importance scores in descending order
rf_sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print feature importance scores for Random Forest
print("Random Forest Feature Importance:")
for feature, importance in rf_sorted_feature_importance:
    print(f"{feature}: {importance}")


# Save the best model and results
best_rf_model_info = {'best_params': best_rf_params, 'best_score': best_rf_score, 'rmse': rmse_rf}

# Add the feature importance dictionary to best_rf_model_info
best_rf_model_info['feature_importance'] = feature_importance_dict


best_rf_file = get_absolute_path(
    file_name='best_rf_model.joblib',
    rel_path='results'
)
save_model(best_rf_file, best_rf_model, best_rf_model_info)


/Users/yinpuli/opt/anaconda3/envs/py-311/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/yinpuli/opt/anaconda3/envs/py-311/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/yinpuli/opt/anaconda3/envs/py-311/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/yinpuli/opt/anaconda3/envs/py-311/lib/pyth

KeyboardInterrupt: 

Random Forest Feature Importance:
Dissolved oxygen, water, unfiltered, milligrams per liter (Maximum): 0.8114432607956025
Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Mean): 0.05523080506466742
Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Maximum): 0.03645052629807038
pH, water, unfiltered, field, standard units (Minimum): 0.024152677368460457
Temperature, water, degrees Celsius (Maximum): 0.01893949261600433
Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Minimum): 0.011631621147613309
pH, water, unfiltered, field, standard units (Maximum): 0.0103268770299261
Dissolved oxygen, water, unfiltered, milligrams per liter (Minimum): 0.009883523970297617
Temperature, water, degrees Celsius (Mean): 0.008015312669312775
Temperature, water, degrees Celsius (Minimum): 0.00788698311428718
Dissolved oxygen, water, unfiltered, milligrams per liter (Mean): 0.00603

In [ ]:
best_rf_model_info

{'best_params': {'max_depth': 20,
  'min_samples_leaf': 4,
  'min_samples_split': 5,
  'n_estimators': 100},
 'best_score': 0.011567736990528689,
 'rmse': 0.01148450307834224,
 'feature_importance': {'Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Maximum)': 0.03645052629807038,
  'pH, water, unfiltered, field, standard units (Maximum)': 0.0103268770299261,
  'pH, water, unfiltered, field, standard units (Minimum)': 0.024152677368460457,
  'Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Minimum)': 0.011631621147613309,
  'Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius (Mean)': 0.05523080506466742,
  'Dissolved oxygen, water, unfiltered, milligrams per liter (Maximum)': 0.8114432607956025,
  'Dissolved oxygen, water, unfiltered, milligrams per liter (Mean)': 0.006038919925758083,
  'Dissolved oxygen, water, unfiltered, milligrams per liter (Minimum)': 0.0

In [ ]:
best_rf_model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
# import shap

# # Initialize a SHAP explainer
# rf_explainer = shap.Explainer(best_rf_model)

# # Calculate SHAP values for a set of data (e.g., dval)
# rf_shap_values = rf_explainer.shap_values(dval)

# # Create a summary plot of feature importances using SHAP
# shap.summary_plot(rf_shap_values, dval, plot_type="bar")
